In [1]:
!nvidia-smi

Mon Jul 31 21:27:41 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 531.68                 Driver Version: 531.68       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                      TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1660 T...  WDDM | 00000000:01:00.0 Off |                  N/A |
| N/A   50C    P8                7W /  N/A|      0MiB /  6144MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [20]:
!pip install ultralytics
!pip install roboflow
from ultralytics import YOLO
import matplotlib.pyplot as plt

# model = YOLO('yolov8n.pt')
# model.predict(
#    source='https://media.roboflow.com/notebooks/examples/dog.jpeg',
#    conf=0.25
# )


In [3]:
# download annotated dataset

from roboflow import Roboflow
rf = Roboflow(api_key="MCsuPF4jcnNKM8ueqwPN")
project = rf.workspace("testtrain").project("checx")
dataset = project.version(5).download("yolov8")


loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.134 is required but found version=8.0.145, to fix: `pip install ultralytics==8.0.134`


Extracting Dataset Version Zip to Checx-5 in yolov8:: 100%|██████████| 336/336 [00:00<00:00, 1170.12it/s]


In [4]:
dataset.location

'd:\\MSResearch\\Coding\\Legacy-Contracts-Transformer\\Checx-5'

In [6]:
# !yolo task=detect mode=train model=yolov8s.pt data={dataset.location}/data.yaml epochs=100 imgsz=640

^C


Ultralytics YOLOv8.0.20  Python-3.10.11 torch-2.0.1+cpu CPU
yolo\engine\trainer: task=detect, mode=train, model=yolov8s.yaml, data=d:\MSResearch\Coding\Legacy-Contracts-Transformer\Checx-5/data.yaml, epochs=100, patience=50, batch=16, imgsz=640, save=True, cache=False, device=, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, overlap_mask=True, mask_ratio=4, dropout=False, val=True, save_json=False, save_hybrid=False, conf=0.001, iou=0.7, max_det=300, half=False, dnn=False, plots=False, source=ultralytics/assets/, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, int8=False, dynamic=False, si

In [10]:

!pip install -q transformers

In [11]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
import requests
from PIL import Image

processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-handwritten")



IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-handwritten and are newly initialized: ['encoder.pooler.dense.weight', 'encoder.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
# load image from the IAM dataset
url = "https://fki.tic.heia-fr.ch/static/img/a01-122-02.jpg"
image = Image.open(requests.get(url, stream=True).raw).convert("RGB")

pixel_values = processor(image, return_tensors="pt").pixel_values
generated_ids = model.generate(pixel_values)

generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [14]:
generated_text

'industry, " Mr. Brown commented icily. " Let us have a'

In [19]:
model.save_pretrained(save_directory=".\\TrOCR-Processor\\base-handwritten")